In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

/home/den/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('big_table.csv', sep=',')

In [3]:
data[25:35]

,Unnamed: 0,drug_cod,drug_x,provider_code,provider_drug_cod,code,drug_y
25,25,9953,Ацетилсалициловая к-та 500мг №10 тб,0000000000000000000013561,5163,13561,"Ацетилсалициловая к-та таб 0,5 г №10"
26,26,10604,Сульфокамфокаин 100мг/мл 2мл №10 амп р-р д/ин,0000000000000000000010250,5163,10250,Сульфокамфокаин амп 2 мл №10
27,27,11805,Пирацетам 1200мг №20 тб пл/о,0000000000000000000017469,5163,17469,Пирацетам таб 1200 мг № 20 п/о
28,28,3941,Торвакард 20мг №30 тб пл/о,0000000000000000000017256,5163,17256,"Торвакард 0,02 №30 таб. п\о"
29,29,6214,Фенистил гель 0.1% 50г (тубы),0000000000000000000015263,5163,15263,"Фенистил 0,1% гель 50 г"
30,30,5111,Бетасерк 24мг №20 тб,0000000000000000000014629,5163,14629,Бетасерк таб 24 мг №20
31,31,5132,Дибазол - УБФ 20мг №10 тб,0000000000000000000008789,5163,8789,Дибазол таб 20 мг №10
32,32,6229,Клотримазол 100мг №6 тб ваг,0000000000000000000010395,5163,10395,Клотримазол таб ваг 100 мг №6
33,33,4145,"Мидокалм Рихтер 100мг+2,5мг/мл 1мл №5 амп р-р...",0000000000000000000014400,5163,14400,"Мидокалм-Рихтер амп 100 мг+2,5 мг/мл 1 мл №5"
34,34,6191,Гевискон форте (мята) 150мл №1 сусп фл,0000000000000000000018330,5163,18330,Гевискон форте сусп фл 150 мл мятный


In [4]:
Parma_Sentence = data.drug_x.tolist()
Parma_drug_cod = data.drug_cod.tolist()
Other_Sentence = data.drug_y.tolist()
Other_Sentence = ['B' + ''.join(DrugName)  + 'E' for DrugName in Other_Sentence]
Parma_Sentence = ['B' + ''.join(DrugName)  + 'E' for DrugName in Parma_Sentence]
print('Количество строк в списке Парма Медикал:', len(Parma_Sentence), sep = ' ')
print('Количество строк в списке сторонних организаций:', len(Other_Sentence), sep = ' ')
PM_unik_drug = set()
for drug_line in Parma_drug_cod:
    if (drug_line not in PM_unik_drug):
        PM_unik_drug.add(drug_line)
PM_unik_drug = sorted(list(PM_unik_drug))    
PM_unik_drug = set(PM_unik_drug)
print('Количество уникальных препаратов Парма Медикал:',len(PM_unik_drug), sep = ' ')

Количество строк в списке Парма Медикал: 20206
Количество строк в списке сторонних организаций: 20206
Количество уникальных препаратов Парма Медикал: 7230


In [5]:
print(Parma_Sentence[1225:1235], Other_Sentence[1225:1235], sep = '\n')

['B Пентоксифиллин 100мг №60 тб п/о кшчр E', 'B Валериана Форте 40мг №50 тб пл/оE', 'B Луцетам 200мг/мл 5мл №10 р-р д/в/м и в/в введенияE', 'B Амитриптилин 25мг №50 тбE', 'B Брусники листья №20 1,5г ф/пE', 'B Тест-полоски  Аккучек Перформа №50 E', 'B Дексалгин 25мг №10 тб пл/о E', 'B Гастал №30 тб д/рассE', 'B Гастал №30 тб д/рассE', 'B Белогент 15г №1 крем для нар прим тубаE']
['BПентоксифиллин таб.п/к/о 0,1 №60 ФармпроектE', 'BВалериана форте таб. п.о 40мг №50  E', 'BЛуцетам р-р в/в и в/м 200мг/мл 5мл №10  E', 'BАмитриптилин таб. 25мг №50  E', 'BБрусника листья 1,5г №20  E', 'BАкку-чек перформа тест-полоски д/глюкометра №50  E', 'BДексалгин таб. п.о 25мг №10  E', 'BГастал таб. д/рассас №30  E', 'BГастал таб. д/рассас №30  E', 'BБелогент крем 15г  E']


In [6]:
PM_chars = set()
for drug_line in Parma_Sentence:
    for ch in drug_line:
        if (ch not in PM_chars):
            PM_chars.add(ch)
PM_chars = sorted(list(PM_chars))    
PM_chars = set(PM_chars)
display(len(PM_chars))

141

In [7]:
Other_chars = set()
display(type(Other_chars))
for drug_line in Other_Sentence:
    for ch in drug_line:
        if (ch not in Other_chars):
            Other_chars.add(ch)
Other_chars = sorted(list(Other_chars))
Other_chars = set(Other_chars)
display(len(Other_chars))

set

151

In [8]:
#display(PM_chars, Other_chars)

In [9]:
display('Сравнение словарей:')
display('Нет в словаре ПМ:')
display(Other_chars - PM_chars)
display('Нет во внешнем словаре:')
display(type(PM_chars))
display(type(Other_chars))
delta = PM_chars - Other_chars
display(delta)
display('Дилемма - добавлять ли во внешние словари символы ПМ? Они же могут там рано или поздно появиться.')
display('А, давайте-ка добавим!')
for ch in delta:
    Other_chars.add(ch)
display('Проверкочка:')
delta = PM_chars - Other_chars
display("Количество симовлов, которых нет во внешнем словаре:" + str(len(delta)))

'Сравнение словарей:'

'Нет в словаре ПМ:'

{'#', '=', '@', 'J', '\\', 'j', 'z', '\xa0', 'Щ', 'Ъ', 'Ы', 'Ь', '–'}

'Нет во внешнем словаре:'

set

...

{'&', '<', '>'}

'Дилемма - добавлять ли во внешние словари символы ПМ? Они же могут там рано или поздно появиться.'

'А, давайте-ка добавим!'

'Проверкочка:'

'Количество симовлов, которых нет во внешнем словаре:0'

In [10]:
PM_index_to_char_dict = {}
PM_char_to_index_dict = {}

for k, v in enumerate(PM_chars):
    PM_index_to_char_dict[k] = v
    PM_char_to_index_dict[v] = k

In [11]:
Other_index_to_char_dict = {}
Other_char_to_index_dict = {}

for k, v in enumerate(Other_chars):
    Other_index_to_char_dict[k] = v
    Other_char_to_index_dict[v] = k

In [12]:
max_len_PM_sent = max([len(line) for line in Parma_Sentence])
max_len_Other_sent = max([len(line) for line in Other_Sentence])

In [13]:
display(max_len_PM_sent)
display(max_len_Other_sent)

103

109

In [14]:
nb_samples = 20206
#nb_samples = 1000
tokenized_Other_sentences = np.zeros(shape = (nb_samples,max_len_Other_sent,len(Other_chars)), dtype='float32')
#print(tokenized_Other_sentences)

In [15]:
tokenized_PM_sentences = np.zeros(shape = (nb_samples,max_len_PM_sent,len(PM_chars)), dtype='float32')
#print(tokenized_PM_sentences)

In [16]:
target_data = np.zeros((nb_samples, max_len_PM_sent, len(PM_chars)),dtype='float32')
#print(target_data)

In [17]:
for i in range(nb_samples):
    for k,ch in enumerate(Other_Sentence[i]):
        tokenized_Other_sentences[i,k,Other_char_to_index_dict[ch]] = 1
        
    for k,ch in enumerate(Parma_Sentence[i]):
        tokenized_PM_sentences[i,k,PM_char_to_index_dict[ch]] = 1

        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i,k-1,PM_char_to_index_dict[ch]] = 1

In [18]:
encoder_input = Input(shape=(None,len(Other_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]

In [19]:
decoder_input = Input(shape=(None,len(PM_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(PM_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)

In [20]:
decoder_out.shape

TensorShape([Dimension(None), Dimension(None), Dimension(141)])

In [21]:
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x=[tokenized_Other_sentences,tokenized_PM_sentences], 
          y=target_data,
          batch_size=164,
          epochs=120,
          validation_split=0.2)

Train on 16164 samples, validate on 4042 samples
Epoch 1/120
16164/16164 [==============================] - 302s 19ms/step - loss: 1.0711 - val_loss: 0.9831
Epoch 2/120
16164/16164 [==============================] - 307s 19ms/step - loss: 0.9217 - val_loss: 0.7576
Epoch 3/120
16164/16164 [==============================] - 418s 26ms/step - loss: 0.7418 - val_loss: 0.6606
Epoch 4/120
16164/16164 [==============================] - 561s 35ms/step - loss: 0.6550 - val_loss: 0.5983
Epoch 5/120
16164/16164 [==============================] - 582s 36ms/step - loss: 0.6071 - val_loss: 0.5540
Epoch 6/120
16164/16164 [==============================] - 513s 32ms/step - loss: 0.5687 - val_loss: 0.5271
Epoch 7/120
16164/16164 [==============================] - 458s 28ms/step - loss: 0.5430 - val_loss: 0.5157
Epoch 8/120
16164/16164 [==============================] - 449s 28ms/step - loss: 0.5312 - val_loss: 0.4831
Epoch 9/120
16164/16164 [==============================] - 395s 24ms/step - loss: 0.504

Epoch 76/120
16164/16164 [==============================] - 327s 20ms/step - loss: 0.1352 - val_loss: 0.1572
Epoch 77/120
16164/16164 [==============================] - 462s 29ms/step - loss: 0.1341 - val_loss: 0.1608
Epoch 78/120
16164/16164 [==============================] - 321s 20ms/step - loss: 0.1329 - val_loss: 0.1585
Epoch 79/120
16164/16164 [==============================] - 322s 20ms/step - loss: 0.1323 - val_loss: 0.1562
Epoch 80/120
16164/16164 [==============================] - 322s 20ms/step - loss: 0.1310 - val_loss: 0.1573
Epoch 81/120
16164/16164 [==============================] - 322s 20ms/step - loss: 0.1301 - val_loss: 0.1548
Epoch 82/120
16164/16164 [==============================] - 322s 20ms/step - loss: 0.1291 - val_loss: 0.1565
Epoch 83/120
16164/16164 [==============================] - 427s 26ms/step - loss: 0.1286 - val_loss: 0.1553
Epoch 84/120
16164/16164 [==============================] - 320s 20ms/step - loss: 0.1275 - val_loss: 0.1520
Epoch 85/120
16164/

In [22]:
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [23]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(PM_chars)))
    target_seq[0, 0,PM_char_to_index_dict[' ']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    while not stop_condition:
        
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_PM_char = PM_index_to_char_dict[max_val_index]
        translated_sent += sampled_PM_char
        
        if ( (sampled_PM_char == '\n') or (len(translated_sent) > max_len_PM_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(PM_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
        
    return translated_sent

In [24]:
for seq_index in range(30):
    inp_seq = tokenized_Other_sentences[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-')
    print('Input sentence:', Other_Sentence[seq_index])
    print('Decoded sentence:', translated_sent)

-
Input sentence: BИмудон таб д/рассас №24E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BГлаумакс 0,005% 2,5 мл фл-кап гл каплиE
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BЭссенциале-форте Н капс 300 мг №30.E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BЭссенциале-форте Н капс 300 мг №30.E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BДимедрол таб №10E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BИбупрофен гель 5% 50г тубаE
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,00

In [25]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("my_model.json", "w")
json_file.write(model_json)
json_file.close()

/home/den/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2364: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [26]:
model.save_weights('my_model.h5')

In [29]:
for seq_index in range(60,90):
    inp_seq = tokenized_Other_sentences[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-')
    print('Input sentence:', Other_Sentence[seq_index])
    print('Decoded sentence:', translated_sent)

-
Input sentence: BВанкорус (Ванкомицин) фл 1 г в/в (инд уп)E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BНексиум таб 40 мг №28E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BТизалуд таб 2 мг №30E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BЦетиризин таб 10 мг №10E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BАнтигриппин шип таб №10 грейпфрутE
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEEEEЛЛТ,,,000,0,000000 1мафлин д/ростиома вледли
-
Input sentence: BАмикацин фл 1000 мг в/м в/в (для стационаров)E
Decoded sentence:  Коделак Бронхо №10 тбEEфиптEEкиылипекиEEапеличечичерикEEE

In [30]:
from datetime import datetime
datetime.now()

datetime.datetime(2018, 3, 21, 2, 32, 41, 388344)